**Question**: GPS and other camera information are encoded in exif tags. Can this data be used to determine characteristics such as leaf size? Can other data, such as soil maps, weather, etc. be used to find correlations among phenotypes?

In [19]:
import cv2
import random
import exif
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Monthly

Load the data

In [6]:
HOME_DIR = str(Path().cwd().parent)

SHEET_DATA = pd.read_excel(f"{HOME_DIR}/step3/data.xlsx")
SHEET_DATA.tail()

,filename,treatment,block,row,position,genotype,leaf_color,leaf_shape,brown_splotches
1667,91F395A8-E156-416C-84E8-A2880F067945.jpg,D*,NaN,NaN,NaN,NaN,dark_green,lanceolate,low
1668,18E8E0DC-35EE-4C34-839A-71E0E6D7F3AD.jpg,C,1.0,12.0,27.0,GW-6962_WP,yellow_green,lanceolate,low
1669,DB2D48CC-17CD-4A96-BC52-CFFC5879641B.jpg,C,2.0,5.0,49.0,GW-9784_LM,light_green,lanceolate,none
1670,2258AE77-D53C-446F-8DAD-0E948854EBDC.jpg,C,2.0,42.0,43.0,"**GW-4579_WP**,core",dark_green,lanceolate,low
1671,5FCC7048-CC04-40DA-B3DB-F6404CA41931.jpg,C,2.0,38.0,31.0,BESC-217_LM,dark_green,elliptical,high


Function and type declarations

In [4]:
cv2.Image = np.ndarray
cv2.Contour = np.ndarray

# shows an image on ipynb
def imshow(img: np.ndarray) -> None:
    plt.axis('off')
    plt.grid(False)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

# get a random filename from within the sheet
def get_random_filename() -> str:
    return SHEET_DATA['filename'][random.randint(0, len(SHEET_DATA)-1)]

# get contours that are optimized by the area - to - boudning rect metric, with large contours favored
def get_optimized_contours(contours: list[cv2.Contour], n: int = 1) -> list[cv2.Contour]:
    """ minimize the area of the rectangle and maximize contour area """
    adjusted_contours = []
    for contour in contours:
        _, _, w, h = cv2.boundingRect(contour)
        adjusted_contours.append([contour, cv2.contourArea(contour)**1.5/(w*h)])
    return [x[0] for x in sorted(adjusted_contours, key=lambda x: x[1], reverse=True)[:n]]

# display filename and image, and get image going to filename
def get_image(filename: str = None, show: bool = True):
    if filename is None:
        filename = get_random_filename()
    img = cv2.imread(f"{HOME_DIR}/dataset/{filename}")
    if show:
        print(filename)
        imshow(img)
    return img

Get the EXIF tags from the image

In [33]:
def get_lat_long(filename: str) -> tuple:
    with open(f"{HOME_DIR}/dataset/{filename}", "rb") as f:
        my_image = exif.Image(f)
    return my_image.gps_latitude, my_image.gps_longitude
    lat = my_image.gps_latitude[0] * ((my_image.gps_latitude_ref == 'N') * 2 - 1)
    long = my_image.gps_longitude[0] * ((my_image.gps_longitude_ref == 'E') * 2 - 1)
    return (lat, long)

def get_date(filename: str) -> datetime:
    with open(f"{HOME_DIR}/dataset/{filename}", "rb") as f:
        my_image = exif.Image(f)
    return datetime(*map(int, my_image.gps_datestamp.split('-')))

filename = get_random_filename()
print(get_lat_long(filename))
print(get_date(filename))

((38.0, 32.0, 29.86675), (121.0, 47.0, 19.7549))
2021-09-03 00:00:00


In [36]:
rain_array = []
for filename in SHEET_DATA['filename']:


    # Create Point
    date = get_date(filename)
    rain_array.append(get_lat_long(filename))
    continue
    location = Point(*get_lat_long(filename))

    # Get daily data for 2018
    data = Monthly(location, date - timedelta(days=365), date)
    data = data.fetch()
    rain_array.append(sum([*data.prcp]))

In [37]:
set(rain_array)

{((38.0, 32.0, 28.42915), (121.0, 47.0, 22.042)),
 ((38.0, 32.0, 29.9142), (121.0, 47.0, 22.9167)),
 ((38.0, 32.0, 28.6258), (121.0, 47.0, 21.92138)),
 ((38.0, 32.0, 30.46006), (121.0, 47.0, 23.28676)),
 ((38.0, 32.0, 45.6076), (121.0, 47.0, 32.614)),
 ((38.0, 32.0, 29.03366), (121.0, 47.0, 23.30503)),
 ((38.0, 32.0, 31.338), (121.0, 47.0, 23.39056)),
 ((38.0, 32.0, 29.18144), (121.0, 47.0, 24.80133)),
 ((38.0, 32.0, 31.2649), (121.0, 47.0, 20.39617)),
 ((38.0, 32.0, 30.886), (121.0, 47.0, 20.85977)),
 ((38.0, 32.0, 31.3118), (121.0, 47.0, 19.4945)),
 ((38.0, 32.0, 40.4893), (121.0, 47.0, 33.3608)),
 ((38.0, 32.0, 30.87823), (121.0, 47.0, 21.7682)),
 ((38.0, 32.0, 28.8188), (121.0, 47.0, 22.04196)),
 ((38.0, 32.0, 30.3415), (121.0, 47.0, 19.2632)),
 ((38.0, 32.0, 27.8332), (121.0, 47.0, 20.04454)),
 ((38.0, 32.0, 46.2263), (121.0, 47.0, 32.7263)),
 ((38.0, 32.0, 31.6474), (121.0, 47.0, 23.9104)),
 ((38.0, 32.0, 31.6861), (121.0, 47.0, 19.25677)),
 ((38.0, 32.0, 27.243), (121.0, 47.0, 2